## **Deep Learning for Causal Inference by Vikas Ramachandra**

Applying the concepts from the "Deep Learning for Causal Inference" paper authored by Vikas Ramachandra to the data_for_churn_analysis dataset

link to the paper: <a>https://arxiv.org/abs/1803.00149</a>

In [28]:
# DA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ML
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Metrics
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error


In [29]:
# Loading dataset
df = pd.read_csv('data_for_churn_analysis.csv')

In [30]:
df.shape

(104143, 18)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104143 entries, 0 to 104142
Data columns (total 18 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   device                                 104025 non-null  object 
 1   first_payment_amount                   104143 non-null  int64  
 2   age                                    104001 non-null  float64
 3   city                                   98301 non-null   object 
 4   number_of_cards                        103671 non-null  float64
 5   payments_initiated                     103671 non-null  float64
 6   payments_failed                        103671 non-null  float64
 7   payments_completed                     103671 non-null  float64
 8   payments_completed_amount_first_7days  103671 non-null  float64
 9   reward_purchase_count_first_7days      80879 non-null   float64
 10  coins_redeemed_first_7days             103671 non-null  

In [32]:
# null values?
print(f"{df.isnull().sum().sum()=}")
print(f"perc of dataset missing {df.isnull().sum().sum()/df.shape[0]}")

df.isnull().sum().sum()=37490
perc of dataset missing 0.3599857887712088


In [33]:
# proportion of people who have churned
df['is_churned'].value_counts(normalize=True)

is_churned
0    0.713192
1    0.286808
Name: proportion, dtype: float64

### 1. **Impact of Referrals on Customer Acquisition and Retention**:
   - Research Question: Do customers acquired through referrals (`is_referral`) exhibit different behaviors and retention rates compared to non-referred customers?
   - Treatment: Customer acquisition through referrals
   - Outcome: Customer behavior (e.g., `payments_initiated`, `payments_completed`, `visits_feature_1`, `visits_feature_2`) and churn (`is_churned`)
   - Potential Confounders: `device`, `age`, `city`, `number_of_cards`, `payments_failed`, `payments_completed_amount_first_7days`, `reward_purchase_count_first_7days`, `coins_redeemed_first_7days`, `given_permission_1`, `given_permission_2`

$$Y_i = f(T_i, X_i, \epsilon_i)$$

Where:

- $Y_i$ represents the outcome variable `is_churned` for customer $i$
- $T_i$ is the treatment variable `is_referral`, indicating whether customer $i$ was acquired through a referral
- $X_i$ represents the vector of potential confounding variables for customer $i$, such as `device`, `age`, `city`, `number_of_cards`, `payments_failed`, `payments_completed_amount_first_7days`, `reward_purchase_count_first_7days`, `coins_redeemed_first_7days`, `given_permission_1`, `given_permission_2`
- $\epsilon_i$ is the error term, accounting for unobserved factors affecting the outcome
- $f$ is an unknown function that maps the treatment, confounders, and error term to the outcome

In [34]:
# -------------- DATA PREPROCESSING --------------------

# Encoding using label encoding technique
obj_cols = df.select_dtypes(include='object').columns # grabs object dtypes columns
le = LabelEncoder() # creates LabelEncoder instance
for col in obj_cols:
    df[col] = le.fit_transform(df[col]) # encodes each column

df['is_referral'] = np.where(df['is_referral'] == True, 1, 0)


# -------------- IMPUTING MISSING VALUES --------------------
missing_cols = df.columns[df.isna().any()].tolist()
for col in missing_cols:
    df[col] = df[col].fillna(df[col].mean())


# -------------- DATA SPLIT --------------------

confounders = [
    'device',
    'age',
    'city',
    'number_of_cards',
    'payments_failed',
    'payments_completed_amount_first_7days',
    'reward_purchase_count_first_7days',
    'coins_redeemed_first_7days',
    'given_permission_1',
    'given_permission_2',
    'is_referral' # treatment
]

y = df['is_churned']
X = df[confounders]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
Y_train, Y_test = train_test_split(y, test_size=0.2, random_state=42)

### 1. **Generalized Neighbor Matching using Autoencoders**

The paper proposes using autoencoders, a type of deep neural network, for dimensionality reduction while preserving the local neighborhood structure of the data. This is useful for generalized neighbor matching to estimate individual treatment effects (ITEs).

The key points are:

* In high dimensions, traditional neighbor matching methods like k-nearest neighbors struggle
* Autoencoders can learn a low-dimensional representation that captures the manifold structure
* This low-dimensional encoding preserves local neighborhoods for accurate neighbor identification
* Experiments show autoencoders outperform methods like manifold learning for ITE estimation

In [35]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [36]:
input_dim = X.shape[1]  # Features count

# Input layer
input_layer = Input(shape=(input_dim, ))

# Encoder: Reduce dimensionality
encoded_1 = Dense(64, activation='relu')(input_layer)
encoded_2 = Dense(32, activation='relu')(encoded_1)
encoded_3 = Dense(16, activation='relu')(encoded_2)
encoded_4 = Dense(8, activation='relu')(encoded_3)

# Decoder: Reconstruct the input
decoded_1 = Dense(16, activation='relu')(encoded_4)
decoded_2 = Dense(32, activation='relu')(decoded_1)
decoded_3 = Dense(64, activation='relu')(decoded_2)
decoded_4 = Dense(input_dim, activation='linear')(decoded_3)  # Final output layer

# Autoencoder model
autoencoder = Model(inputs=input_layer, outputs=decoded_4)  # Ensure to output from the last decoding layer
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [37]:
autoencoder.fit(X_train, Y_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))


Epoch 1/50
326/326 [==============================] - 3s 4ms/step - loss: 0.2252 - val_loss: 18916.6152
Epoch 2/50
326/326 [==============================] - 1s 3ms/step - loss: 0.1641 - val_loss: 18919.7578
Epoch 3/50
326/326 [==============================] - 1s 3ms/step - loss: 0.1580 - val_loss: 18917.5625
Epoch 4/50
326/326 [==============================] - 2s 5ms/step - loss: 0.1555 - val_loss: 18914.0742
Epoch 5/50
326/326 [==============================] - 2s 6ms/step - loss: 0.1544 - val_loss: 18915.3496
Epoch 6/50
326/326 [==============================] - 1s 3ms/step - loss: 0.1525 - val_loss: 18914.1914
Epoch 7/50
326/326 [==============================] - 1s 3ms/step - loss: 0.1521 - val_loss: 18918.8926
Epoch 8/50
326/326 [==============================] - 1s 3ms/step - loss: 0.1518 - val_loss: 18919.3516
Epoch 9/50
326/326 [==============================] - 1s 3ms/step - loss: 0.1506 - val_loss: 18918.0723
Epoch 10/50
326/326 [==============================] - 1s 3ms/st

In [38]:
encoder = Model(inputs=input_layer, outputs=decoded_4)
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)
X_encoded = encoder.predict(X)

3255/3255 [==============================] - 6s 2ms/step


In [39]:
mae = tf.keras.metrics.mean_absolute_error(X_test, X_test_encoded)
print("Mean Absolute Error: ", mae.numpy().mean())

Mean Absolute Error:  47.731796


### 2. **Deep Neural Networks (DNNs) for Propensity Score Matching**

Propensity score matching is a popular technique, but traditionally uses logistic regression for propensity score estimation. The paper proposes using deep neural network classifiers instead, presenting a model called PropensityNet.

The key points are:

* DNNs can potentially capture complex non-linear relationships better than logistic regression
* PropensityNet is trained to estimate propensity scores as a binary classification problem
* Experiments show PropensityNet outperforms logistic regression for propensity score estimation
* This leads to better matching of treated and untreated units for ITE calculation

In [40]:
# Regular Logistic Regression
log_reg = LogisticRegression(max_iter = 10000).fit(X_train_encoded, Y_train)
preds = log_reg.predict_proba(X_encoded)

df_copy = df.copy()
df_copy.loc[:, 'propensity_score'] = preds[:, 1]

In [41]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y, preds[:,1])
print(f"Mean Absolute Misclassification Error: {mae}")

Mean Absolute Misclassification Error: 0.2873118659774708


In [42]:
threshold = 0.5
y_pred = (preds >= threshold).astype(int)

# True labels
y_true = y  # Assuming y_test contains the true labels

# Misassignments
misassignments = (np.sum(y_pred[:,1] != y_true)/len(y_true))*100
print(f"Number of Misassignments percentage: {misassignments}")


Number of Misassignments percentage: 19.511633043027377


In [43]:
df_copy.groupby('is_churned')['propensity_score'].mean()

is_churned
0    0.201751
1    0.499927
Name: propensity_score, dtype: float64

In [44]:
# assuming a 0.5 threshold
accuracy_score(y, (preds[:, 1] > 0.5).astype('int'))

0.8048836695697262

In [45]:
X.shape

(104143, 11)

In [46]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, ReLU
from tensorflow.keras.optimizers import Adam

# Define the model using the Sequential API
model = Sequential([
    Dense(64, input_shape=(11,), activation='relu'),  # Input layer with 11 features and first hidden layer with 64 neurons
    Dense(64, activation='relu'),  # Second hidden layer
    Dense(64, activation='relu'),  # Third hidden layer
    Dense(32, activation='relu'),  # Fourth hidden layer
    Dense(16, activation='relu'),  # Fifth hidden layer
    Dropout(0.3),  # Dropout layer
    Dense(1, activation='sigmoid')  # Output layer
])

# Compile the model with binary cross-entropy loss and the Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])  # Include accuracy if you want to monitor it during training


In [47]:
# Train the model
history = model.fit(X_train_encoded, Y_train, epochs=50)  # Specify the number of epochs according to your training needs

Epoch 1/50
2604/2604 [==============================] - 8s 3ms/step - loss: 0.4728 - accuracy: 0.8020
Epoch 2/50
2604/2604 [==============================] - 7s 3ms/step - loss: 0.4641 - accuracy: 0.8032
Epoch 3/50
2604/2604 [==============================] - 9s 3ms/step - loss: 0.4631 - accuracy: 0.8033
Epoch 4/50
2604/2604 [==============================] - 6s 2ms/step - loss: 0.4631 - accuracy: 0.8025
Epoch 5/50
2604/2604 [==============================] - 7s 3ms/step - loss: 0.4631 - accuracy: 0.8025
Epoch 6/50
2604/2604 [==============================] - 6s 2ms/step - loss: 0.4632 - accuracy: 0.8022
Epoch 7/50
2604/2604 [==============================] - 7s 3ms/step - loss: 0.4625 - accuracy: 0.8025
Epoch 8/50
2604/2604 [==============================] - 6s 2ms/step - loss: 0.4630 - accuracy: 0.8023
Epoch 9/50
2604/2604 [==============================] - 7s 3ms/step - loss: 0.4622 - accuracy: 0.8024
Epoch 10/50
2604/2604 [==============================] - 6s 2ms/step - loss: 0.462

---

In [48]:
preds_dnn = model.predict(X_encoded)

3255/3255 [==============================] - 5s 1ms/step


In [49]:
print(preds_dnn)

[[0.81726485]
 [0.904208  ]
 [0.382405  ]
 ...
 [0.38969007]
 [0.13030174]
 [0.08892549]]


In [50]:
preds_dnn = np.squeeze(preds_dnn)
print(preds_dnn.shape)

(104143,)


In [51]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Assuming y_pred_probs are the predicted probabilities and y_true are the actual labels  # Output of the model's sigmoid function
y_true = y                       # Actual labels

# Calculate MAE
mae = mean_absolute_error(y_true, preds_dnn)
print(f"Mean Absolute Classification Error: {mae}")


Mean Absolute Classification Error: 0.3039754900891989


In [52]:
threshold = 0.5
y_pred = (preds_dnn >= threshold).astype(int)
# True labels
y_true = np.array(y) # Assuming y_test contains the true labels
# Misassignments
misassignments = (np.sum(y_pred != y_true)/len(y_true))*100
print(f"Number of Misassignments percentage: {misassignments}")

Number of Misassignments percentage: 19.524115879127738


In [53]:
# assuming a 0.5 threshold
accuracy_score(y, (preds_dnn > 0.5).astype('int'))

0.8047588412087227

In [54]:
pip install keras-tuner

In [55]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras_tuner import HyperModel, RandomSearch

def build_model(hp):
    model = Sequential([
        Dense(units=hp.Int('units_1', min_value=32, max_value=128, step=32),
              input_shape=(11,),
              activation='relu'),
        Dense(units=hp.Int('units_2', min_value=32, max_value=128, step=32),
              activation='relu'),
        Dense(units=hp.Int('units_3', min_value=32, max_value=128, step=32),
              activation='relu'),
        Dense(units=hp.Int('units_4', min_value=16, max_value=64, step=16),
              activation='relu'),
        Dense(units=hp.Int('units_5', min_value=16, max_value=64, step=16),
              activation='relu'),
        Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [56]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory='propnet_dnn_tuning',
    project_name='Deep_Learning'
)

# Optional: Define a callback to clear the training outputs at the end of every training step.
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        from IPython.display import clear_output
        clear_output(wait=True)

In [57]:
tuner.search(x=X_train_encoded, y=Y_train,
             epochs=50,
             batch_size=256,
             validation_data=(X_test_encoded, Y_test),
             callbacks=[ClearTrainingOutput()])


Trial 20 Complete [00h 01m 22s]
val_accuracy: 0.806279718875885

Best val_accuracy So Far: 0.8064237236976624
Total elapsed time: 00h 33m 45s


In [58]:
best_model = tuner.get_best_models(num_models=1)[0]
loss, accuracy = best_model.evaluate(X_test_encoded, Y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

651/651 [==============================] - 1s 2ms/step - loss: 0.4527 - accuracy: 0.8064
Loss: 0.4527221620082855, Accuracy: 0.8064237236976624


In [62]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters found:")
for param, value in best_hyperparameters.values.items():
    print(f"{param}: {value}")

Best hyperparameters found:
units_1: 96
units_2: 96
units_3: 64
units_4: 16
units_5: 32
dropout: 0.1
learning_rate: 0.01


In [60]:
print(best_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 96)                1152      
                                                                 
 dense_1 (Dense)             (None, 96)                9312      
                                                                 
 dense_2 (Dense)             (None, 64)                6208      
                                                                 
 dense_3 (Dense)             (None, 16)                1040      
                                                                 
 dense_4 (Dense)             (None, 32)                544       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 3

In [63]:
# Running the best model using the Sequential API
model = Sequential([
    Dense(96, input_shape=(11,), activation='relu'),  # Input layer with 11 features and first hidden layer with 64 neurons
    Dense(96, activation='relu'),  # Second hidden layer
    Dense(64, activation='relu'),  # Third hidden layer
    Dense(16, activation='relu'),  # Fourth hidden layer
    Dense(32, activation='relu'),  # Fifth hidden layer
    Dropout(0.1),  # Dropout layer
    Dense(1, activation='sigmoid')  # Output layer
])

# Compile the model with binary cross-entropy loss and the Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])  # Include accuracy if you want to monitor it during training

In [64]:
# Train the model
history = model.fit(X_train_encoded, Y_train, epochs=50)  # Specify the number of epochs according to your training needs

Epoch 1/50
2604/2604 [==============================] - 10s 3ms/step - loss: 0.4653 - accuracy: 0.8015
Epoch 2/50
2604/2604 [==============================] - 7s 3ms/step - loss: 0.4613 - accuracy: 0.8025
Epoch 3/50
2604/2604 [==============================] - 9s 4ms/step - loss: 0.4608 - accuracy: 0.8020
Epoch 4/50
2604/2604 [==============================] - 10s 4ms/step - loss: 0.4607 - accuracy: 0.8033
Epoch 5/50
2604/2604 [==============================] - 11s 4ms/step - loss: 0.4600 - accuracy: 0.8030
Epoch 6/50
2604/2604 [==============================] - 12s 5ms/step - loss: 0.4597 - accuracy: 0.8029
Epoch 7/50
2604/2604 [==============================] - 9s 3ms/step - loss: 0.4594 - accuracy: 0.8028
Epoch 8/50
2604/2604 [==============================] - 9s 3ms/step - loss: 0.4599 - accuracy: 0.8025
Epoch 9/50
2604/2604 [==============================] - 8s 3ms/step - loss: 0.4603 - accuracy: 0.8028
Epoch 10/50
2604/2604 [==============================] - 6s 2ms/step - loss: 0

In [65]:
preds_dnn = model.predict(X_encoded)

3255/3255 [==============================] - 5s 1ms/step


In [66]:
print(preds_dnn)

[[0.89484686]
 [0.9537522 ]
 [0.35540637]
 ...
 [0.37305328]
 [0.11928537]
 [0.09210102]]


In [67]:
preds_dnn = np.squeeze(preds_dnn)
print(preds_dnn.shape)

(104143,)


In [68]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Assuming y_pred_probs are the predicted probabilities and y_true are the actual labels  # Output of the model's sigmoid function
y_true = y                       # Actual labels

# Calculate MAE
mae = mean_absolute_error(y_true, preds_dnn)
print(f"Mean Absolute Classification Error: {mae}")


Mean Absolute Classification Error: 0.2831783819328123


In [69]:
threshold = 0.5
y_pred = (preds_dnn >= threshold).astype(int)
# True labels
y_true = np.array(y) # Assuming y_test contains the true labels
# Misassignments
misassignments = (np.sum(y_pred != y_true)/len(y_true))*100
print(f"Number of Misassignments percentage: {misassignments}")

Number of Misassignments percentage: 19.533718060743404


In [70]:
# assuming a 0.5 threshold
accuracy_score(y, (preds_dnn > 0.5).astype('int'))

0.804662819392566

# Deep Learning for Causal Inference

This README summarizes the key points and contributions of the paper "Deep Learning for Causal Inference" by Vikas Ramachandra.

## 1. Generalized Neighbor Matching using Autoencoders

The paper proposes using autoencoders, a type of deep neural network, for dimensionality reduction while preserving the local neighborhood structure of the data. This is useful for generalized neighbor matching to estimate individual treatment effects (ITEs).

The key points are:

* In high dimensions, traditional neighbor matching methods like k-nearest neighbors struggle
* Autoencoders can learn a low-dimensional representation that captures the manifold structure
* This low-dimensional encoding preserves local neighborhoods for accurate neighbor identification
* Experiments show autoencoders outperform methods like manifold learning for ITE estimation

## 2. Deep Neural Networks (DNNs) for Propensity Score Matching

Propensity score matching is a popular technique, but traditionally uses logistic regression for propensity score estimation. The paper proposes using deep neural network classifiers instead, presenting a model called PropensityNet.

The key points are:

* DNNs can potentially capture complex non-linear relationships better than logistic regression
* PropensityNet is trained to estimate propensity scores as a binary classification problem
* Experiments show PropensityNet outperforms logistic regression for propensity score estimation
* This leads to better matching of treated and untreated units for ITE calculation

## Overall Contribution

The paper argues that deep learning models like autoencoders and DNNs can improve upon traditional methods for neighbor matching and propensity score estimation, two crucial steps in causal inference for estimating individual and average treatment effects.

The main contributions are introducing these deep learning approaches, evaluating them on simulated datasets, and demonstrating their potential advantages over existing techniques through experimental results.